In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

ticker = 'AAPL'  # 这里以苹果公司股票为例
data = yf.download(ticker, start='2020-01-01', end='2023-01-01',proxy='http://127.0.0.1:10203')

In [ ]:
data['Return'] = data['Close'].pct_change()
data.dropna(inplace=True)
X = data[['Open', 'High', 'Low', 'Close', 'Volume']]
y = data['Return']

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
train_size = int(len(X_scaled) * 0.8)
X_train, X_test = X_scaled[:train_size], X_scaled[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data['Close'][:train_size], label='True values (Train)')
plt.plot(range(train_size, train_size + len(train_predict)), train_predict, label='Predicted values (Train)')
plt.plot(range(train_size, len(data['Close'])), data['Close'][train_size:], label='True values (Test)')
plt.plot(range(train_size + len(train_predict), len(data['Close']) + len(train_predict) - len(y_test)), test_predict, label='Predicted values (Test)')
plt.legend()
plt.show()